In [35]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols, glm
import re

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
pwd

'/content'

### 데이터 확인 - clinical 상위 체계

In [40]:
data = pd.read_csv("./drive/MyDrive/DW_metformin/DQM A Concept Sets.csv") 
cdc= data[data['Class']=='Clinical Drug Comp']
cdg= data[data['Class']=='Clinical Dose Group'] #병용 성분 표기 
cdf= data[data['Class']=='Clinical Drug Form']  #병용 성분 표기 

In [41]:
data.head()

,Unnamed: 0,Id,Code,Name,Class,RC,DRC,Distance,Domain,Vocabulary
0,NaN,36223144,1161610,metformin Oral Product,Clinical Dose Group,0,"803,830",1,Drug,RxNorm
1,NaN,36223145,1161611,metformin Pill,Clinical Dose Group,0,"803,830",1,Drug,RxNorm
2,NaN,40164903,860974,metformin hydrochloride 500 MG,Clinical Drug Comp,0,"673,389",1,Drug,RxNorm
3,NaN,40063354,372803,metformin Oral Tablet,Clinical Drug Form,0,"573,231",1,Drug,RxNorm
4,NaN,40063356,368254,metformin Oral Tablet [Glucophage],Branded Drug Form,0,"462,354",1,Drug,RxNorm


In [ ]:
#CDG - 성분만 추출, 용량은 없음. 
import re
cdg_name =cdg['Name']
cdg_ingredient = [re.sub("metformin|Oral|Product|Pill|/|Liquid", "",i) for i in cdg_name] #특정 문자 제거 
cdg_ingredient= [i.strip() for i in cdg_ingredient] #공백제거 
cdg_ingredient = list(set(cdg_ingredient))
print(cdg_ingredient)

['',
 'dapagliflozin',
 'chlorpropamide',
 'glyburide',
 'glipizide',
 'canagliflozin',
 'pioglitazone',
 'ertugliflozin',
 'sitagliptin',
 'ropinirole',
 'linagliptin',
 'saxagliptin',
 'alogliptin',
 'rosiglitazone',
 'dapagliflozin    saxagliptin',
 'empagliflozin',
 'repaglinide',
 'empagliflozin  linagliptin']

In [ ]:
# CDF-성분만 추출 
cdf_name = cdf['Name']
cdf_ingredient = [re.sub("Powder|Solution|for|metformin|Metformin|Oral|Tablet|Suspension|Injectable Solution|Capsule|Extended Release|/|Delayed Release", "",i) for i in cdf_name] #특정 문자 제거 
cdf_ingredient= [i.strip() for i in cdf_ingredient] #공백제거 
cdf_ingredient = list(set(cdf_ingredient))
cdf_ingredient

['',
 'dapagliflozin',
 'teneligliptin',
 'glipizide',
 'pioglitazone',
 'saxagliptin',
 'alogliptin',
 'Disintegrating',
 'voglibose',
 'repaglinide',
 'Linagliptin',
 'rosuvastatin',
 'sitagliptin',
 'ropinirole',
 'dapagliflozin    saxagliptin',
 'Glyburide',
 'Mitiglinide Calcium hydrate',
 'lobeglitazone',
 'gemigliptin',
 'glimepiride',
 'empagliflozin  linagliptin',
 'chlorpropamide',
 'glyburide',
 'ertugliflozin',
 'linagliptin',
 'atorvastatin',
 'anagliptin',
 'vildagliptin',
 'evogliptin',
 '24 Hour',
 'Gliclazide',
 'nateglinide',
 'canagliflozin',
 '12 hour',
 'rosiglitazone',
 'empagliflozin']

### 데이터 확인 - Branded-하위체계
#### Branded Drug Comp --> 성분명,용량?`
#### Branded Drug Form --> 성분명

In [ ]:
# Branded Drug Comp 
# 복합제 성분만 우선 추출: / 가 있는 것만 추출 
# 숫자, MG, 상품명 제거 []안에 든 문자 모두 제거 
# metformin 제거  
bdc= data[data['Class']=='Branded Drug Comp']
bdc.head()

,Unnamed: 0,Id,Code,Name,Class,Standard Concept Caption,RC,DRC,Distance,Domain,Vocabulary
342,NaN,40164932,860976,metformin hydrochloride 500 MG [Glucophage],Branded Drug Comp,Standard,0,"370,637",1.0,Drug,RxNorm
439,NaN,42943100,OMOP4669731,Metformin 500 MG [DIABEX XR],Branded Drug Comp,Standard,0,"142,452",1.0,Drug,RxNorm Extension
984,NaN,42943169,OMOP4669801,Metformin 850 MG [GLUPA],Branded Drug Comp,Standard,0,"71,027",1.0,Drug,RxNorm Extension
987,NaN,42943002,OMOP4669636,Metformin 250 MG [Glucophage],Branded Drug Comp,Standard,0,"65,168",1.0,Drug,RxNorm Extension
989,NaN,42953874,OMOP4680427,glimepiride 2 MG / Metformin 500 MG [AMARYL-MEX],Branded Drug Comp,Standard,0,"53,543",1.0,Drug,RxNorm Extension


### 병용 약물 확인

In [ ]:
## 병용 성분 확인 (branded drug comp) 
# branded drug comp 우선, labeling 
ingredient = re.compile('(?P<ingredient1>[a-zA-Z]+\s?[a-zA-Z]+) [0-9]+? MG? /? (?P<ingredient2>[a-zA-Z]+\s?[a-zA-Z]+)? [0-9]+? MG? [[a-zA-Z]+]?')
matchobj= [ingredient.search(i)  for i in bdc["Name"]] # 복합제 및 병용만 추출  
ing=list(filter(None, matchobj))  # none값 필터링


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Possible nested set at position 107
  after removing the cwd from sys.path.


In [ ]:
## 병용 성분 확인 
ingredient1= [i.group(1) for i in ing]
ingredient1=list(set(ingredient1)) #중복제거 
ingredient2= [i.group(2) for i in ing]
ingredient2=list(set(ingredient2)) #중복제거 
total = ingredient1+ ingredient2
total = list(set(total))
total

' gnd: repaglinide, nateglinide\n    sglt-2: -ozin\n    statin: -statin\n    su: gl-\n    dpp4i: -liptin\n    tzd: -zone\n'

## 조건식으로 분류 
### 병용 성분 확인: 
#####clinical drug form (성분/성분/제형, 3제요법도 포함), 
#####clinical dose group (성분/성분 제형)
#####branded drug comp (성분 용량mg / 성분 용량mg [상품명])
#####branded drug form(성분/ 성분 제형 [상품명])

    gnd: -glinide
    sglt-2: -ozin
    alpha : -bose
    su: gl- or -amide
    dpp4i: -liptin
    tzd: -zone

    ** 제거  statin: -statin, ropinirole (치매약)


In [ ]:
# 병용 약물군 분류 
## 성분/성분 형식만: clinical drug form, clinical dose group, branded drug  form 
condition= (data.Class=='Clinical Drug Form') | (data.Class=='Branded Drug Form') |(data.Class=='Clinical Dose Group')
db1= data[condition]
db1.head()

,Unnamed: 0,Id,Code,Name,Class,Standard Concept Caption,RC,DRC,Distance,Domain,Vocabulary
6,NaN,36223144,1161610,metformin Oral Product,Clinical Dose Group,Classification,0,"803,830",1.0,Drug,RxNorm
7,NaN,36223145,1161611,metformin Pill,Clinical Dose Group,Classification,0,"803,830",1.0,Drug,RxNorm
156,NaN,40063354,372803,metformin Oral Tablet,Clinical Drug Form,Standard,0,"573,231",1.0,Drug,RxNorm
336,NaN,40063356,368254,metformin Oral Tablet [Glucophage],Branded Drug Form,Standard,0,"462,354",1.0,Drug,RxNorm
360,NaN,40063350,372804,metformin Extended Release Oral Tablet,Clinical Drug Form,Standard,0,"230,599",1.0,Drug,RxNorm


In [77]:
## 병용 성분 갯수 세기: 3제 요법이 있는지 확인. 
db1['ingredient_count'] =   [i.count(' / ')+1 for i in db1['Name']]

,Unnamed: 0,Id,Code,Name,Class,Standard Concept Caption,RC,DRC,Distance,Domain,Vocabulary,type,ingredient_count
6,NaN,36223144,1161610,metformin Oral Product,Clinical Dose Group,Classification,0,"803,830",1.0,Drug,RxNorm,metformin,1
7,NaN,36223145,1161611,metformin Pill,Clinical Dose Group,Classification,0,"803,830",1.0,Drug,RxNorm,metformin,1
156,NaN,40063354,372803,metformin Oral Tablet,Clinical Drug Form,Standard,0,"573,231",1.0,Drug,RxNorm,metformin,1
336,NaN,40063356,368254,metformin Oral Tablet [Glucophage],Branded Drug Form,Standard,0,"462,354",1.0,Drug,RxNorm,metformin,1
360,NaN,40063350,372804,metformin Extended Release Oral Tablet,Clinical Drug Form,Standard,0,"230,599",1.0,Drug,RxNorm,metformin,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17202,NaN,43710540,OMOP937201,Metformin Oral Tablet [Metformin Dura],Branded Drug Form,Standard,0,0,1.0,Drug,RxNorm Extension,metformin,1
17451,NaN,35749642,OMOP2751103,Metformin Delayed Release Oral Tablet [Glucoph...,Branded Drug Form,Non-Standard,0,0,1.0,Drug,RxNorm Extension,metformin,1
17452,NaN,40894299,OMOP2092261,Metformin Oral Tablet [Metformin Lichtenstein],Branded Drug Form,Standard,0,0,1.0,Drug,RxNorm Extension,metformin,1
17453,NaN,40894298,OMOP2092260,Metformin Oral Tablet [Metformin Puren],Branded Drug Form,Standard,0,0,1.0,Drug,RxNorm Extension,metformin,1


In [ ]:
##2제 요법만 골라서 분류 
db2= db1[db1.ingredient_count==2]
db2.head()

In [86]:
ingredient = re.compile('(?P<ingredient1>[a-zA-Z]+) / (?P<ingredeint2>[a-zA-Z]+)')
matchobj= [ingredient.search(i)  for i in db2["Name"]]  
matchobj
lists=[]
for v in matchobj:
  if  v.group(1).lower().startswith('gl') or v.group(2).lower().startswith('gl'):
    lists.append('SU')
  elif v.group(1).lower().endswith('amide') or v.group(2).lower().endswith('amide'):
    lists.append('SU')
  elif v.group(1).lower().endswith('ozin') or v.group(2).lower().endswith('ozin'):
    lists.append('sglt2')
  elif v.group(1).lower().endswith('liptin') or v.group(2).lower().endswith('liptin'):
    lists.append('dpp4i')
  elif v.group(1).lower().endswith('zone') or v.group(2).lower().endswith('zone'):
    lists.append('tzd')
  elif v.group(1).lower().endswith('glinide') or v.group(2).lower().endswith('glinide'):
    lists.append('gnd')
  elif v.group(1).lower().endswith('bose') or v.group(2).lower().endswith('bose'):
    lists.append('alpha')
  else:
    lists.append('error')
lists
db2['type']=lists
db2.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Unnamed: 0,Id,Code,Name,Class,Standard Concept Caption,RC,DRC,Distance,Domain,Vocabulary,type,ingredient_count
992,NaN,42954009,OMOP4680559,glimepiride / Metformin Extended Release Oral ...,Branded Drug Form,Standard,0,"53,543",1.0,Drug,RxNorm Extension,SU,2
993,NaN,42954005,OMOP4680555,glimepiride / Metformin Extended Release Oral ...,Clinical Drug Form,Standard,0,"53,543",1.0,Drug,RxNorm Extension,SU,2
994,NaN,36223141,1161607,metformin / sitagliptin Oral Product,Clinical Dose Group,Classification,0,"51,835",1.0,Drug,RxNorm,dpp4i,2
995,NaN,36223142,1161608,metformin / sitagliptin Pill,Clinical Dose Group,Classification,0,"51,835",1.0,Drug,RxNorm,dpp4i,2
998,NaN,42953926,OMOP4680477,glimepiride / Metformin Oral Tablet,Clinical Drug Form,Standard,0,"42,157",1.0,Drug,RxNorm Extension,SU,2


In [87]:
db2.to_csv('./test.csv')

In [90]:
# 1제 요법은 type= single
# 3제 요법은 type = error 
db11=db1[db1.ingredient_count ==1]
db13=db1[db1.ingredient_count ==3]
db11['type'] = 'single'
db13['type'] = 'error'


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [94]:
#1,2,3 제 요법 합치기. 
db = pd.concat([db11, db2, db13], axis=0)
db.head()

,Unnamed: 0,Id,Code,Name,Class,Standard Concept Caption,RC,DRC,Distance,Domain,Vocabulary,type,ingredient_count
6,NaN,36223144,1161610,metformin Oral Product,Clinical Dose Group,Classification,0,"803,830",1.0,Drug,RxNorm,single,1
7,NaN,36223145,1161611,metformin Pill,Clinical Dose Group,Classification,0,"803,830",1.0,Drug,RxNorm,single,1
156,NaN,40063354,372803,metformin Oral Tablet,Clinical Drug Form,Standard,0,"573,231",1.0,Drug,RxNorm,single,1
336,NaN,40063356,368254,metformin Oral Tablet [Glucophage],Branded Drug Form,Standard,0,"462,354",1.0,Drug,RxNorm,single,1
360,NaN,40063350,372804,metformin Extended Release Oral Tablet,Clinical Drug Form,Standard,0,"230,599",1.0,Drug,RxNorm,single,1


In [97]:
# branded drug comp (성분 용량mg / 성분 용량mg [상품명]) 분류 
## ingredient_count 병용약물 갯수 세기 
db1= data[data.Class=='Branded Drug Comp']
db1['ingredient_count'] =   [i.count(' / ')+1 for i in db1['Name']]
##2제 요법만 골라서 분류 
db2= db1[db1.ingredient_count==2]
db2.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Unnamed: 0,Id,Code,Name,Class,RC,DRC,Distance,Domain,Vocabulary,ingredient_count
18,NaN,42953874,OMOP4680427,glimepiride 2 MG / Metformin 500 MG [AMARYL-MEX],Branded Drug Comp,0,"53,543",1,Drug,RxNorm Extension,2
37,NaN,42953853,OMOP4680406,glimepiride 2 MG / Metformin 500 MG [AMARYL M],Branded Drug Comp,0,"24,525",1,Drug,RxNorm Extension,2
38,NaN,40164924,861820,metformin hydrochloride 500 MG / sitagliptin 5...,Branded Drug Comp,0,"20,852",1,Drug,RxNorm,2
39,NaN,42953726,OMOP4680281,glimepiride 1 MG / Metformin 250 MG [AMARYL M],Branded Drug Comp,0,"17,632",1,Drug,RxNorm Extension,2
40,NaN,36884530,OMOP995178,Metformin 850 MG / sitagliptin 50 MG [Janumet],Branded Drug Comp,0,"16,371",1,Drug,RxNorm Extension,2


In [102]:
db2.head(7)

,Unnamed: 0,Id,Code,Name,Class,RC,DRC,Distance,Domain,Vocabulary,ingredient_count
18,NaN,42953874,OMOP4680427,glimepiride 2 MG / Metformin 500 MG [AMARYL-MEX],Branded Drug Comp,0,"53,543",1,Drug,RxNorm Extension,2
37,NaN,42953853,OMOP4680406,glimepiride 2 MG / Metformin 500 MG [AMARYL M],Branded Drug Comp,0,"24,525",1,Drug,RxNorm Extension,2
38,NaN,40164924,861820,metformin hydrochloride 500 MG / sitagliptin 5...,Branded Drug Comp,0,"20,852",1,Drug,RxNorm,2
39,NaN,42953726,OMOP4680281,glimepiride 1 MG / Metformin 250 MG [AMARYL M],Branded Drug Comp,0,"17,632",1,Drug,RxNorm Extension,2
40,NaN,36884530,OMOP995178,Metformin 850 MG / sitagliptin 50 MG [Janumet],Branded Drug Comp,0,"16,371",1,Drug,RxNorm Extension,2
47,NaN,42942971,OMOP4669605,Metformin 1000 MG / sitagliptin 100 MG [JANUME...,Branded Drug Comp,0,"14,427",1,Drug,RxNorm Extension,2
53,NaN,2057616,OMOP4958521,glyburide 2.5 MG / metformin 400 MG [GLIBOMET],Branded Drug Comp,0,"11,782",1,Drug,RxNorm Extension,2


In [149]:
ingredient = re.compile('(?P<ingredient1>[a-zA-Z]+[ ]?[a-zA-Z]+?) \d{0,}[.]?\d{0,}? MG / (?P<ingredeint2>[a-zA-Z]+[ ]?[a-zA-Z]+?[ ]?[a-zA-Z]+?) \d{0,}[.]?\d{0,}? MG')
matchobj= [ingredient.search(i)  for i in db2["Name"]]  
matchobj

lists=[]
for v in matchobj:
  if v == None:
    lists.append('error')
  elif  v.group(1).lower().startswith('gl') or v.group(2).lower().startswith('gl'):
    lists.append('SU')
  elif v.group(1).lower().endswith('amide') or v.group(2).lower().endswith('amide'):
    lists.append('SU')
  elif v.group(1).lower().endswith('ozin') or v.group(2).lower().endswith('ozin'):
    lists.append('sglt2')
  elif v.group(1).lower().endswith('liptin') or v.group(2).lower().endswith('liptin'):
    lists.append('dpp4i')
  elif v.group(1).lower().endswith('zone') or v.group(2).lower().endswith('zone'):
    lists.append('tzd')
  elif v.group(1).lower().endswith('glinide') or v.group(2).lower().endswith('glinide'):
    lists.append('gnd')
  elif v.group(1).lower().startswith('mitiglinide') or v.group(2).lower().startswith('mitiglinide'): #mitiglinide calcium hydrate를 제거하기 위함. 
    lists.append('gnd') 
  elif v.group(1).lower().endswith('bose') or v.group(2).lower().endswith('bose'):
    lists.append('alpha')
  else:
    lists.append('error')
lists
db2['type']=lists
db2.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Unnamed: 0,Id,Code,Name,Class,RC,DRC,Distance,Domain,Vocabulary,ingredient_count,type
18,NaN,42953874,OMOP4680427,glimepiride 2 MG / Metformin 500 MG [AMARYL-MEX],Branded Drug Comp,0,"53,543",1,Drug,RxNorm Extension,2,SU
37,NaN,42953853,OMOP4680406,glimepiride 2 MG / Metformin 500 MG [AMARYL M],Branded Drug Comp,0,"24,525",1,Drug,RxNorm Extension,2,SU
38,NaN,40164924,861820,metformin hydrochloride 500 MG / sitagliptin 5...,Branded Drug Comp,0,"20,852",1,Drug,RxNorm,2,dpp4i
39,NaN,42953726,OMOP4680281,glimepiride 1 MG / Metformin 250 MG [AMARYL M],Branded Drug Comp,0,"17,632",1,Drug,RxNorm Extension,2,SU
40,NaN,36884530,OMOP995178,Metformin 850 MG / sitagliptin 50 MG [Janumet],Branded Drug Comp,0,"16,371",1,Drug,RxNorm Extension,2,dpp4i


In [150]:
db2.to_csv('test.csv')

In [151]:
# 1제 요법은 type= single
# 3제 요법은 type = error 
db11=db1[db1.ingredient_count ==1]
db13=db1[db1.ingredient_count ==3]
db11['type'] = 'single'
db13['type'] = 'error'


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [152]:
#1,2,3 제 요법 합치기. 
dbdb = pd.concat([db11, db2, db13], axis=0)
dbdb.head()

,Unnamed: 0,Id,Code,Name,Class,RC,DRC,Distance,Domain,Vocabulary,ingredient_count,type
7,NaN,40164932,860976,metformin hydrochloride 500 MG [Glucophage],Branded Drug Comp,0,"370,637",1,Drug,RxNorm,1,single
11,NaN,42943100,OMOP4669731,Metformin 500 MG [DIABEX XR],Branded Drug Comp,0,"142,452",1,Drug,RxNorm Extension,1,single
15,NaN,42943169,OMOP4669801,Metformin 850 MG [GLUPA],Branded Drug Comp,0,"71,027",1,Drug,RxNorm Extension,1,single
17,NaN,42943002,OMOP4669636,Metformin 250 MG [Glucophage],Branded Drug Comp,0,"65,168",1,Drug,RxNorm Extension,1,single
23,NaN,42943016,OMOP4669650,Metformin 1000 MG [DIABEX XR],Branded Drug Comp,0,"43,200",1,Drug,RxNorm Extension,1,single


In [154]:
# 합치기 
total = pd.concat([dbdb, db], axis=0)
total.head()

,Unnamed: 0,Id,Code,Name,Class,RC,DRC,Distance,Domain,Vocabulary,ingredient_count,type,Standard Concept Caption
7,NaN,40164932,860976,metformin hydrochloride 500 MG [Glucophage],Branded Drug Comp,0,"370,637",1.0,Drug,RxNorm,1,single,NaN
11,NaN,42943100,OMOP4669731,Metformin 500 MG [DIABEX XR],Branded Drug Comp,0,"142,452",1.0,Drug,RxNorm Extension,1,single,NaN
15,NaN,42943169,OMOP4669801,Metformin 850 MG [GLUPA],Branded Drug Comp,0,"71,027",1.0,Drug,RxNorm Extension,1,single,NaN
17,NaN,42943002,OMOP4669636,Metformin 250 MG [Glucophage],Branded Drug Comp,0,"65,168",1.0,Drug,RxNorm Extension,1,single,NaN
23,NaN,42943016,OMOP4669650,Metformin 1000 MG [DIABEX XR],Branded Drug Comp,0,"43,200",1.0,Drug,RxNorm Extension,1,single,NaN


In [ ]:
db.head()

In [ ]:
dbdb.head()

In [157]:
# 필요한 부분만 추출
adm_type =total[['Id', 'Code', 'Name', 'Class', 'type','ingredient_count']]

In [158]:
adm_type.head()

,Id,Code,Name,Class,type,ingredient_count
7,40164932,860976,metformin hydrochloride 500 MG [Glucophage],Branded Drug Comp,single,1
11,42943100,OMOP4669731,Metformin 500 MG [DIABEX XR],Branded Drug Comp,single,1
15,42943169,OMOP4669801,Metformin 850 MG [GLUPA],Branded Drug Comp,single,1
17,42943002,OMOP4669636,Metformin 250 MG [Glucophage],Branded Drug Comp,single,1
23,42943016,OMOP4669650,Metformin 1000 MG [DIABEX XR],Branded Drug Comp,single,1


In [159]:
adm_type.to_csv("./drive/MyDrive/DW_metformin/adm_type.csv")